In [1]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()


True

In [2]:
class ProjectRecommender:
    """
    Recommends relevant project ideas for missing skills using LLM.
    """
    
    def __init__(self, api_key=None):
        """
        Initialize the ProjectRecommender with API credentials.
        
        Args:
            api_key (str): OpenAI API key. If None, reads from environment variable.
        """
        self.api_key = api_key or os.getenv('OPENAI_API_KEY')
        if not self.api_key:
            raise ValueError("OpenAI API key not provided. Set OPENAI_API_KEY environment variable.")
        
        self.client = OpenAI(api_key=self.api_key)
    
    def recommend_projects(self, missing_skills, num_projects=3):
        """
        Generate project recommendations for missing skills.
        
        Args:
            missing_skills (list): List of skills that need to be demonstrated
            num_projects (int): Number of projects to recommend per skill (default: 3)
            
        Returns:
            dict: Dictionary with skills as keys and project recommendations as values
        """
        if not missing_skills:
            return {"message": "No missing skills provided"}
        
        # Create prompt for LLM
        prompt = self._create_prompt(missing_skills, num_projects)
        
        try:
            # Call OpenAI API
            response = self.client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "You are a technical career advisor who specializes in portfolio building and project-based learning for software developers and tech professionals."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.8,
                max_tokens=2500
            )
            
            # Parse the response
            recommendations = self._parse_response(response.choices[0].message.content)
            return recommendations
            
        except Exception as e:
            return {"error": f"Failed to generate recommendations: {str(e)}"}
    
    def _create_prompt(self, missing_skills, num_projects):
        """
        Create a detailed prompt for the LLM.
        
        Args:
            missing_skills (list): List of missing skills
            num_projects (int): Number of projects to recommend
            
        Returns:
            str: Formatted prompt
        """
        skills_str = ", ".join(missing_skills)
        
        prompt = f"""
I need project ideas that would help demonstrate proficiency in these skills: {skills_str}

For each skill, suggest {num_projects} portfolio-worthy project ideas that would effectively showcase this skill to potential employers.

For each project recommendation, provide:
1. Project Title (creative and descriptive)
2. Difficulty Level (Beginner, Intermediate, Advanced)
3. Estimated Time to Complete (in hours or days)
4. Description (3-4 sentences about what the project does)
5. Key Features (list 3-5 main features to implement)
6. Skills Demonstrated (list the specific skills this project showcases)
7. Technologies/Tools Needed (list technologies required)
8. Portfolio Impact (1-2 sentences on why this project would impress employers)
9. Bonus Challenges (optional advanced features to make it stand out)

Project Guidelines:
- Projects should be realistic and completable
- Should result in something tangible to show on GitHub/portfolio
- Mix of different difficulty levels when appropriate
- Should solve a real problem or have practical use
- Should be impressive enough to discuss in interviews

Format your response as a JSON object with this structure:
{{
    "skill_name": [
        {{
            "title": "Project Title",
            "difficulty": "Beginner/Intermediate/Advanced",
            "estimated_time": "X hours/days",
            "description": "Project description",
            "key_features": ["feature1", "feature2", "feature3"],
            "skills_demonstrated": ["skill1", "skill2", "skill3"],
            "technologies": ["tech1", "tech2", "tech3"],
            "portfolio_impact": "Why this impresses employers",
            "bonus_challenges": ["challenge1", "challenge2"]
        }}
    ]
}}

Only return valid JSON, no additional text.
"""
        return prompt
    
    def _parse_response(self, response_text):
        """
        Parse the LLM response into structured format.
        
        Args:
            response_text (str): Raw response from LLM
            
        Returns:
            dict: Parsed recommendations
        """
        try:
            # Try to extract JSON from the response
            start_idx = response_text.find('{')
            end_idx = response_text.rfind('}') + 1
            
            if start_idx != -1 and end_idx != 0:
                json_str = response_text[start_idx:end_idx]
                recommendations = json.loads(json_str)
                return recommendations
            else:
                return {"error": "Could not parse JSON from response", "raw_response": response_text}
                
        except json.JSONDecodeError as e:
            return {"error": f"JSON parsing error: {str(e)}", "raw_response": response_text}
    
    def save_recommendations(self, recommendations, filename="project_recommendations.json"):
        """
        Save recommendations to a JSON file.
        
        Args:
            recommendations (dict): Project recommendations
            filename (str): Output filename
        """
        try:
            with open(filename, 'w') as f:
                json.dump(recommendations, f, indent=4)
            print(f"Recommendations saved to {filename}")
        except Exception as e:
            print(f"Error saving recommendations: {str(e)}")

In [3]:
def main():
    """
    Example usage of ProjectRecommender
    """
    # Example missing skills (you would get these from gap_analysis)
    missing_skills = ["Machine Learning", "React.js", "Docker"]
    
    # Initialize recommender
    recommender = ProjectRecommender()
    
    # Get recommendations
    print("Generating project recommendations...")
    recommendations = recommender.recommend_projects(missing_skills, num_projects=3)
    
    # Print results
    print("\n" + "="*50)
    print("PROJECT RECOMMENDATIONS")
    print("="*50 + "\n")
    
    if "error" in recommendations:
        print(f"Error: {recommendations['error']}")
    else:
        for skill, projects in recommendations.items():
            print(f"\n💡 {skill.upper()}")
            print("-" * 40)
            for i, project in enumerate(projects, 1):
                print(f"\n{i}. {project.get('title', 'N/A')}")
                print(f"   Difficulty: {project.get('difficulty', 'N/A')}")
                print(f"   Time: {project.get('estimated_time', 'N/A')}")
                print(f"   Description: {project.get('description', 'N/A')}")
                print(f"   Key Features:")
                for feature in project.get('key_features', []):
                    print(f"      • {feature}")
                print(f"   Technologies: {', '.join(project.get('technologies', []))}")
                print(f"   Portfolio Impact: {project.get('portfolio_impact', 'N/A')}")
    
    # Save to file
    recommender.save_recommendations(recommendations)


if __name__ == "__main__":
    main()

Generating project recommendations...

PROJECT RECOMMENDATIONS


💡 MACHINE LEARNING
----------------------------------------

1. Predictive Analysis for Stock Trading
   Difficulty: Advanced
   Time: 30 days
   Description: A machine learning project where you predict stock prices using historical data. The predictions can be used for trading stocks.
   Key Features:
      • Historical data analysis
      • Predictive algorithms
      • Graphic visualization of predictions
   Technologies: Python, Scikit-learn, Pandas, Matplotlib/Seaborn
   Portfolio Impact: This project showcases the ability to apply machine learning techniques to real world financial data, a highly valuable skill in many industries.

2. Image Recognition System
   Difficulty: Intermediate
   Time: 20 days
   Description: A machine learning project to identify objects in images. Can be used for cataloging images, self-driving cars, or surveillance systems.
   Key Features:
      • Object identification
      • Image p